In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Research Project

/content/drive/My Drive/Research Project


In [3]:
import pandas as pd
import numpy as np
import os
# import src
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
pip install -U mittens

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from statistics import mean, stdev, median, mode
# With PCA
from sklearn.decomposition import PCA

from sklearn.feature_extraction.text import CountVectorizer
# SVM
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from mittens import GloVe, Mittens

In [23]:
pip install autocorrect

     |████████████████████████████████| 624kB 6.5MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.2.2-cp36-none-any.whl size=621491 sha256=0b19964f7925764d992cc036fb1b9a80fc904a38edde0a5ddb3e49d41c54f03d
  Stored in directory: /root/.cache/pip/wheels/b4/0b/7d/98268d64c8697425f712c897265394486542141bbe4de319d6
Successfully built autocorrect


In [24]:

import nltk
import string
from nltk.tokenize import word_tokenize
import random
import pickle
from nltk.corpus import stopwords

from autocorrect import Speller
# from pycontractions import Contractions

# from spellchecker import SpellChecker

import re
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer 

from hyperopt import fmin, tpe, hp

In [25]:
# nltk.download('all')

In [26]:
from numpy import asarray
from numpy import zeros
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [27]:
# nltk.download('all')


## Getting Data

In [10]:
data_path = r'/content/drive/My Drive/Research Project/Data'

In [11]:
# data_path = r'C:\Users\kalya\OneDrive - University of Illinois at Chicago\!UIC\!Semesters\3rd Sem\CS 583 Data Mining and Text Mining\Research Project\Data'

In [12]:
train_pr_df = pd.read_csv(os.path.join(data_path, 'Combined Training1 Data.csv'), usecols = [1,2])


In [13]:
train_pr_df.head()

,Doc Text,Sentiment
0,debate job come back obama get as make corp ta...,Negative
1,romney romney abrasive,Negative
2,sicken obama war woman tom halloran,Negative
3,mitt romney business genius speculator play bi...,Negative
4,day election day ready cheer president obama t...,Positive


In [14]:
train_pr_df = train_pr_df.dropna()


### Converting data into TF-IDF

In [15]:
def tf_idf(x):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(x)
    #print(tfidf_matrix)
    x1 = tfidf_matrix.toarray()
#     print(x1)
    return x1

In [16]:
train_pr_tfidf = tf_idf(train_pr_df['Doc Text'])
# romney_train_pr_tfidf = tf_idf(romney_train_pr_df['Doc Text'])

In [17]:
train_pr_tfidf.shape

(11117, 10120)

### Cross Valid Function

In [18]:
def cross_valid(X,y,model):
    f1_Positive  =[]
    f1_Neutral =[]
    f1_Negative =[]
    acc =[]
    cv = KFold(n_splits=5,shuffle=True)
    for train_index, val_index in cv.split(X):
    #     print("Train Index: ", train_index, "\n")
    #     print("Test Index: ", test_index)
    #     print(X.iloc[train_index])
    #     print(f)
        # print(1)
        X_train1, X_val, y_train1, y_val = X[train_index], X[val_index], y.iloc[train_index], y.iloc[val_index]
        temp = y_train1.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        model = model.fit(X_train1, temp)

        y_pred = model.predict(X_val)
        temp_y_val = y_val.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        f1_temp = f1_score(temp_y_val, y_pred, average = None)

        f1_Positive.append(f1_temp[0])
        f1_Neutral.append(f1_temp[1])
        f1_Negative.append(f1_temp[2])

        acc.append(round(accuracy_score(temp_y_val, y_pred),3))

    return round(mean(acc),3), round(mean(f1_Positive),3), round(mean(f1_Neutral),3), round(mean(f1_Negative),3)

### Cross Valid func with PCA

In [19]:
def cross_valid_PCA(X,y,model, n):
    f1_Positive  =[]
    f1_Neutral =[]
    f1_Negative =[]
    acc =[]
    cv = KFold(n_splits=5,shuffle=True)
    
    pca = PCA(n_components=n)
    
    
    for train_index, val_index in cv.split(X):
    #     print("Train Index: ", train_index, "\n")
    #     print("Test Index: ", test_index)
    #     print(X.iloc[train_index])
    #     print(f)
        # print(1)
        X_train1, X_val, y_train1, y_val = X[train_index], X[val_index], y.iloc[train_index], y.iloc[val_index]
        x_pca1 = pca.fit_transform(X_train1)
        temp = y_train1.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        model = model.fit(x_pca1, temp)
        
        x_pca_val = pca.transform(X_val)
        y_pred = model.predict(x_pca_val)
        temp_y_val = y_val.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        f1_temp = f1_score(temp_y_val, y_pred, average = None)

        f1_Positive.append(f1_temp[0])
        f1_Neutral.append(f1_temp[1])
        f1_Negative.append(f1_temp[2])

        acc.append(round(accuracy_score(temp_y_val, y_pred),3))

    return round(mean(acc),3), round(mean(f1_Positive),3), round(mean(f1_Neutral),3), round(mean(f1_Negative),3)

### Logistic Regression

In [20]:
from sklearn.linear_model import LogisticRegression

In [21]:


# # clf = SVC(kernel="rbf", gamma=1, class_weight='balanced')
# #     error = 1-(cross_val_score(clf, x_pca1, temp, cv = 5, scoring = 'f1_macro'))
# #     f1 = mean(error) + stdev(error) 
# #print(f1)


# clf = LogisticRegression(class_weight='balanced')
# accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
# eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
# eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
# # print([f1_list[2],f1_list[1],f1_list[0]])

# print(eval_dict)


In [29]:
# With Bayesian Optimization, and LR

def objective_func_LR(args):

    C = args['C']
    # penalty = args['penalty']
    # multi_class = args['multi_class']
    # l1_ratio = args['l1_ratio']

    print(C)
    clf = LogisticRegression(C= C,class_weight='balanced',n_jobs =-1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train_pr_tfidf,train_pr_df['Sentiment'], clf) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print(eval_dict)
    return -(eval_score)


space = {'C': hp.uniform('C', 0,20),
        # 'penalty': hp.choice('penalty', ['l1', 'l2']), 
        # 'multi_class': hp.choice('multi_class',['ovr', 'multinomial']),

        #  'l1_ratio' : hp.uniform('l1_ratio',0,1)
        }                        
                                
                                
best_classifier_LR = fmin(objective_func_LR, space, algo=tpe.suggest, max_evals=10)
print(best_classifier_LR)

8.673530387320984
  0%|          | 0/10 [03:01<?, ?it/s, best loss: ?]


KeyboardInterrupt: ignored

In [ ]:
{'accuracy': 0.57, 'f1_pos': 0.546, 'f1_neu': 0.496, 'f1_neg': 0.639, 'eval_score': 0.585}


In [ ]:
# With PCA, Bayesian Optimization, and LR

def objective_func_LR_PCA(args):

    C = args['C']
    n = args['n']
    
    # penalty = args['penalty']
    # multi_class = args['multi_class']
    # l1_ratio = args['l1_ratio']


    clf = LogisticRegression(C= C,class_weight='balanced',n_jobs =-1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train_pr_tfidf,train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    par_dict = {'C': C, 'n': n}
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print(par_dict)
    print(eval_dict)
    print('\n')    
    return -(eval_score)


space = {'C': hp.uniform('C', 0,2),
         'n': hp.choice('n', np.arange(100,1000, step =100))
        # 'penalty': hp.choice('penalty', ['l1', 'l2']), 
        # 'multi_class': hp.choice('multi_class',['ovr', 'multinomial']),
        #  'l1_ratio' : hp.uniform('l1_ratio',0,1)
        }                        
                                
                                
best_classifier_LR_PCA = fmin(objective_func_LR_PCA, space, algo=tpe.suggest, max_evals=10)
print(best_classifier_LR_PCA)

{'C': 1.8434857486520984, 'n': 300}                   
{'accuracy': 0.541, 'f1_pos': 0.523, 'f1_neu': 0.476, 'f1_neg': 0.604, 'eval_score': 0.556}
{'C': 1.5030580591896512, 'n': 900}                                 
{'accuracy': 0.558, 'f1_pos': 0.54, 'f1_neu': 0.487, 'f1_neg': 0.624, 'eval_score': 0.574}
{'C': 1.2327957870944484, 'n': 700}                                  
{'accuracy': 0.555, 'f1_pos': 0.541, 'f1_neu': 0.484, 'f1_neg': 0.619, 'eval_score': 0.572}
{'C': 1.4260381337579342, 'n': 200}                                  
{'accuracy': 0.545, 'f1_pos': 0.52, 'f1_neu': 0.49, 'f1_neg': 0.604, 'eval_score': 0.556}
{'C': 1.8566388055538825, 'n': 600}                                  
{'accuracy': 0.552, 'f1_pos': 0.532, 'f1_neu': 0.481, 'f1_neg': 0.62, 'eval_score': 0.568}
{'C': 1.401649559841088, 'n': 800}                                   
{'accuracy': 0.554, 'f1_pos': 0.536, 'f1_neu': 0.482, 'f1_neg': 0.62, 'eval_score': 0.57}
{'C': 1.5928811919880783, 'n': 600}               

In [ ]:
# # With Bayesian Optimization, different class weights and LR

# def objective_func_LR(args):

#     C = args['C']
#     balance = args['balance']
#     # multi_class = args['multi_class']
#     # l1_ratio = args['l1_ratio']

 
#     clf = LogisticRegression(C= C,class_weight=balance,n_jobs =-1)
#     accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(romney_train_pr_tfidf,romney_train_pr_df['Sentiment'], clf) 
#     eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
#     eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
#     print(eval_dict)
#     return -(eval_score)


# space = {'C': hp.uniform('C', 0,10),
#          'balance': hp.choice('balance', [{1:3, 2:1, 3:2}, {1:15, 2:1, 3:10}, {1:20, 2:5, 3:10}, {1:100, 2:10, 3:100}])
#         # 'penalty': hp.choice('penalty', ['l1', 'l2']), 
#         # 'multi_class': hp.choice('multi_class',['ovr', 'multinomial']),

#         #  'l1_ratio' : hp.uniform('l1_ratio',0,1)
#         }                        
                                
                                
# best_classifier_LR = fmin(objective_func_LR, space, algo=tpe.suggest, max_evals=20)
# print(best_classifier_LR)

### SVM

In [ ]:
# C = 1
# n = 3
# kernel = 'rbf'

# par_dict = {'C': C, 'n': n, 'kernel': kernel}
# print(par_dict)

# clf = SVC(C= C, kernel = kernel, class_weight='balanced')
# accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train_pr_tfidf,train_pr_df['Sentiment'], clf, n) 
# eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)

# eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
# print(eval_dict)
# print('\n')    
# return -(eval_score)

In [ ]:
# With PCA, Bayesian Optimization, and SVM

def objective_func_SVM_PCA(args):
    C = args['C']
    n = args['n']
    kernel = args['kernel']
#     gamma = args['gamma']
#     degree = args['degree']

    par_dict = {'C': C, 'n': n, 'kernel': kernel}
    print(par_dict)
 
    clf = SVC(C= C, kernel = kernel, class_weight='balanced')
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train_pr_tfidf,train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)

    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print(eval_dict)
    print('\n')    
    return -(eval_score)


space = {'C': hp.uniform('C', 0,1),
         'n': hp.choice('n', np.arange(3, 503, step =50)),
        'kernel': hp.choice('kernel', ['linear', 'rbf']) 
#         'gamma': hp.choice('gamma',range(1,4)),
#          'degree' : hp.choice('degree',range(1,4))
        }
                                
                       
                                
                                
                                
best_classifier_SVM_PCA = fmin(objective_func_SVM_PCA, space, algo=tpe.suggest, max_evals=20)
print(best_classifier_SVM_PCA)

{'C': 0.062326381967102384, 'n': 53, 'kernel': 'linear'}
{'accuracy': 0.505, 'f1_pos': 0.379, 'f1_neu': 0.469, 'f1_neg': 0.586, 'eval_score': 0.49}
{'C': 0.33829254293365374, 'n': 3, 'kernel': 'linear'}             
{'accuracy': 0.45, 'f1_pos': 0.372, 'f1_neu': 0.381, 'f1_neg': 0.537, 'eval_score': 0.453}
{'C': 0.5372641670533641, 'n': 53, 'kernel': 'rbf'}                
{'accuracy': 0.526, 'f1_pos': 0.5, 'f1_neu': 0.451, 'f1_neg': 0.596, 'eval_score': 0.541}
{'C': 0.3622603032203806, 'n': 353, 'kernel': 'rbf'}                
{'accuracy': 0.564, 'f1_pos': 0.533, 'f1_neu': 0.491, 'f1_neg': 0.634, 'eval_score': 0.577}
{'C': 0.7324611346380544, 'n': 103, 'kernel': 'rbf'}                 
{'accuracy': 0.538, 'f1_pos': 0.506, 'f1_neu': 0.479, 'f1_neg': 0.603, 'eval_score': 0.549}
{'C': 0.24875299620690916, 'n': 403, 'kernel': 'rbf'}                
{'accuracy': 0.563, 'f1_pos': 0.528, 'f1_neu': 0.483, 'f1_neg': 0.639, 'eval_score': 0.577}
{'C': 0.49003910749639024, 'n': 3, 'kernel': 'line

### KNN

In [ ]:
# WIth Bayesian Optimization and KNN
def objective_func_KNN(args):
    n_neighbors = args['n_neighbors']
    metric = args['metric']  

    par_dict = {'n_neighbors': n_neighbors, 'metric': metric}
    print(par_dict)
    
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train_pr_tfidf,train_pr_df['Sentiment'], clf) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)

    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_neighbors': hp.choice('n_neighbors',np.arange(2,32, step =3)),
        'metric':hp.choice('metric', ["euclidean","manhattan"])
        }

best_classifier_KNN = fmin(objective_func_KNN, space, algo=tpe.suggest, max_evals=10)
print(best_classifier_KNN)

{'n_neighbors': 26, 'metric': 'manhattan'}            
{'accuracy': 0.354, 'f1_pos': 0.44, 'f1_neu': 0.227, 'f1_neg': 0.302, 'eval_score': 0.365}
{'n_neighbors': 2, 'metric': 'euclidean'}                              
{'accuracy': 0.429, 'f1_pos': 0.47, 'f1_neu': 0.42, 'f1_neg': 0.39, 'eval_score': 0.43}
{'n_neighbors': 5, 'metric': 'manhattan'}                              
{'accuracy': 0.411, 'f1_pos': 0.449, 'f1_neu': 0.39, 'f1_neg': 0.379, 'eval_score': 0.413}
{'n_neighbors': 20, 'metric': 'euclidean'}                            
{'accuracy': 0.51, 'f1_pos': 0.513, 'f1_neu': 0.402, 'f1_neg': 0.582, 'eval_score': 0.535}
{'n_neighbors': 26, 'metric': 'manhattan'}                             
{'accuracy': 0.351, 'f1_pos': 0.435, 'f1_neu': 0.239, 'f1_neg': 0.294, 'eval_score': 0.36}
{'n_neighbors': 8, 'metric': 'euclidean'}                              
{'accuracy': 0.492, 'f1_pos': 0.499, 'f1_neu': 0.419, 'f1_neg': 0.543, 'eval_score': 0.511}
{'n_neighbors': 8, 'metric': 'euclidean'} 

In [ ]:
# WIth PCA , Bayesian Optimization and KNN
def objective_func_KNN_PCA(args):
    n_neighbors = args['n_neighbors']
    metric = args['metric']
    n = args['n']    

    par_dict = {'n_neighbors': n_neighbors, 'n': n, 'metric': metric}
    print(par_dict)
    
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train_pr_tfidf,train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)

    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_neighbors': hp.choice('n_neighbors',np.arange(1,10, step =1)),
        'metric':hp.choice('metric', ["euclidean","manhattan"]),
        'n': hp.choice('n', np.arange(3,123, step =10))}

best_classifier_KNN_PCA = fmin(objective_func_KNN_PCA, space, algo=tpe.suggest, max_evals=15)
print(best_classifier_KNN_PCA)

{'n_neighbors': 8, 'n': 53, 'metric': 'euclidean'}    
{'accuracy': 0.481, 'f1_pos': 0.438, 'f1_neu': 0.441, 'f1_neg': 0.539, 'eval_score': 0.486}
{'n_neighbors': 5, 'n': 23, 'metric': 'manhattan'}                  
{'accuracy': 0.465, 'f1_pos': 0.431, 'f1_neu': 0.417, 'f1_neg': 0.527, 'eval_score': 0.474}
{'n_neighbors': 3, 'n': 33, 'metric': 'euclidean'}                  
{'accuracy': 0.462, 'f1_pos': 0.434, 'f1_neu': 0.387, 'f1_neg': 0.534, 'eval_score': 0.477}
{'n_neighbors': 5, 'n': 43, 'metric': 'manhattan'}                  
{'accuracy': 0.474, 'f1_pos': 0.437, 'f1_neu': 0.43, 'f1_neg': 0.535, 'eval_score': 0.482}
{'n_neighbors': 6, 'n': 93, 'metric': 'euclidean'}                  
{'accuracy': 0.476, 'f1_pos': 0.439, 'f1_neu': 0.439, 'f1_neg': 0.529, 'eval_score': 0.481}
{'n_neighbors': 9, 'n': 63, 'metric': 'manhattan'}                  
{'accuracy': 0.487, 'f1_pos': 0.435, 'f1_neu': 0.446, 'f1_neg': 0.55, 'eval_score': 0.491}
{'n_neighbors': 5, 'n': 73, 'metric': 'manhattan'}

### Naive Bayes

In [ ]:
clf = GaussianNB()
accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train_pr_tfidf,train_pr_df['Sentiment'], clf) 
eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
print(eval_dict)

{'accuracy': 0.387, 'f1_pos': 0.44, 'f1_neu': 0.346, 'f1_neg': 0.36, 'eval_score': 0.396}


In [ ]:
# With PCA and Naive bayes
def objective_func_NB_PCA(args):
    n = args['n']

    par_dict = {'n': n}
    print(par_dict)
    
    clf = GaussianNB()
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train_pr_tfidf,train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n': hp.choice('n', np.arange(3,1023, step =50))}

best_classifier_NB_PCA = fmin(objective_func_NB_PCA, space, algo=tpe.suggest, max_evals=20)



{'n': 153}                                            
{'accuracy': 0.446, 'f1_pos': 0.39, 'f1_neu': 0.478, 'f1_neg': 0.44, 'eval_score': 0.425}
{'n': 253}                                                          
{'accuracy': 0.446, 'f1_pos': 0.404, 'f1_neu': 0.47, 'f1_neg': 0.444, 'eval_score': 0.431}
{'n': 153}                                                          
{'accuracy': 0.448, 'f1_pos': 0.394, 'f1_neu': 0.476, 'f1_neg': 0.447, 'eval_score': 0.43}
{'n': 3}                                                            
{'accuracy': 0.458, 'f1_pos': 0.303, 'f1_neu': 0.175, 'f1_neg': 0.61, 'eval_score': 0.457}
{'n': 553}                                                          
{'accuracy': 0.428, 'f1_pos': 0.416, 'f1_neu': 0.431, 'f1_neg': 0.434, 'eval_score': 0.426}
{'n': 803}                                                          
{'accuracy': 0.388, 'f1_pos': 0.403, 'f1_neu': 0.37, 'f1_neg': 0.389, 'eval_score': 0.393}
{'n': 853}                                            

## Ensemble Methods

### Random Forest

In [ ]:
# With Bayesian Optimization, and Random Forest

def objective_func_RF(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']

    par_dict = {'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train_pr_tfidf,train_pr_df['Sentiment'], clf) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,601, step =20)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
        }
                                
                                
                                
best_classifier_RF = fmin(objective_func_RF, space, algo=tpe.suggest, max_evals=25)
print(best_classifier_RF)

{'n_estimators': 440, 'max_depth': 30, 'min_samples_split': 7, 'min_samples_leaf': 9}
{'accuracy': 0.488, 'f1_pos': 0.255, 'f1_neu': 0.147, 'f1_neg': 0.631, 'eval_score': 0.458}
{'n_estimators': 560, 'max_depth': 72, 'min_samples_split': 9, 'min_samples_leaf': 2}
{'accuracy': 0.548, 'f1_pos': 0.446, 'f1_neu': 0.331, 'f1_neg': 0.663, 'eval_score': 0.552}
{'n_estimators': 40, 'max_depth': 62, 'min_samples_split': 3, 'min_samples_leaf': 9}
{'accuracy': 0.527, 'f1_pos': 0.373, 'f1_neu': 0.289, 'f1_neg': 0.655, 'eval_score': 0.518}
{'n_estimators': 520, 'max_depth': 52, 'min_samples_split': 5, 'min_samples_leaf': 7}
{'accuracy': 0.527, 'f1_pos': 0.384, 'f1_neu': 0.265, 'f1_neg': 0.654, 'eval_score': 0.522}
{'n_estimators': 20, 'max_depth': 74, 'min_samples_split': 3, 'min_samples_leaf': 8}
{'accuracy': 0.53, 'f1_pos': 0.399, 'f1_neu': 0.306, 'f1_neg': 0.655, 'eval_score': 0.528}
{'n_estimators': 280, 'max_depth': 39, 'min_samples_split': 10, 'min_samples_leaf': 1}
{'accuracy': 0.526, 'f1_po

In [ ]:
# With PCA,  Bayesian Optimization, and Random Forest

def objective_func_RF_PCA(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']
    n = args['n']

    par_dict = { 'n': n, 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train_pr_tfidf,train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,601, step =20)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1)),
        'n': hp.choice('n', np.arange(3,203, step =20))
        }
                                
                                
                                
best_classifier_RF_PCA = fmin(objective_func_RF_PCA, space, algo=tpe.suggest, max_evals=30)
print(best_classifier_RF_PCA)

{'n': 163, 'n_estimators': 300, 'max_depth': 53, 'min_samples_split': 4, 'min_samples_leaf': 4}
{'accuracy': 0.55, 'f1_pos': 0.439, 'f1_neu': 0.403, 'f1_neg': 0.657, 'eval_score': 0.549}
{'n': 123, 'n_estimators': 320, 'max_depth': 33, 'min_samples_split': 9, 'min_samples_leaf': 7}
{'accuracy': 0.549, 'f1_pos': 0.443, 'f1_neu': 0.399, 'f1_neg': 0.657, 'eval_score': 0.55}
{'n': 103, 'n_estimators': 300, 'max_depth': 84, 'min_samples_split': 7, 'min_samples_leaf': 4}
{'accuracy': 0.557, 'f1_pos': 0.459, 'f1_neu': 0.424, 'f1_neg': 0.659, 'eval_score': 0.558}
{'n': 123, 'n_estimators': 40, 'max_depth': 94, 'min_samples_split': 3, 'min_samples_leaf': 7}
{'accuracy': 0.546, 'f1_pos': 0.448, 'f1_neu': 0.42, 'f1_neg': 0.648, 'eval_score': 0.547}
{'n': 123, 'n_estimators': 440, 'max_depth': 87, 'min_samples_split': 9, 'min_samples_leaf': 3}
{'accuracy': 0.552, 'f1_pos': 0.454, 'f1_neu': 0.407, 'f1_neg': 0.656, 'eval_score': 0.554}
{'n': 3, 'n_estimators': 580, 'max_depth': 100, 'min_samples_spl

In [ ]:
# # WITH LDA
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from math import sqrt
# lda = LinearDiscriminantAnalysis(n_components = None)
# x_lda = lda.fit_transform(train_pr_tfidf,train_pr_df['Sentiment'])

In [ ]:
# # With LDA, Bayesian Optimization, and Random Forest

# def objective_func(args):
#     n_estimators = args['n_estimators']
#     max_depth = args['max_depth']
#     min_samples_split = args['min_samples_split']
#     min_samples_leaf = args['min_samples_leaf']
# #     pca = PCA(n_components=args['n'])
# #     x_pca1 = pca.fit_transform(obama_train_pr_tfidf)

 
#     clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    
#     temp = obama_train_pr_df['Sentiment'].apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
#     temp_f1 = cross_val_score(clf, x_lda, temp, cv = 5, scoring = 'f1_macro')
#     f1 = mean(temp_f1)
#     return -(f1)
# space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
#          'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
# #         'criterion': hp.choice('criterion',['gini', 'entropy']),
# #         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
#         'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
#          'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
# #         'n': hp.choice('n', np.arange(3,20, step =1))
#         }
                                
                                
                                
# best_classifier = fmin(objective_func, space, algo=tpe.suggest, max_evals=30)
# print(best_classifier)# With Bayesian Optimization, and Random Forest



In [ ]:
# obama_train2_pr_tfidf, obama_val_pr_tfidf, obama_train2_pr_class, obama_val_pr_class = train_test_split(obama_train_pr_tfidf, obama_train_pr_df['Sentiment'], test_size = 0.2)

In [ ]:
# x_lda2 = lda.fit_transform(train_pr_tfidf,train_pr_df['Sentiment'])

In [ ]:
# # With LDA, Bayesian Optimization, and Random Forest

# def objective_func(args):
#     n_estimators = args['n_estimators']
#     max_depth = args['max_depth']
#     min_samples_split = args['min_samples_split']
#     min_samples_leaf = args['min_samples_leaf']
# #     pca = PCA(n_components=args['n'])
# #     x_pca1 = pca.fit_transform(obama_train_pr_tfidf)

 
#     clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    
# #     temp = obama_train_pr_df['Sentiment'].apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
#     temp_f1 = cross_val_score(clf, x_lda2, obama_train2_pr_class, cv = 5, scoring = 'f1_macro')
#     f1 = mean(temp_f1)
#     return -(f1)
# space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
#          'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
# #         'criterion': hp.choice('criterion',['gini', 'entropy']),
# #         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
#         'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
#          'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
# #         'n': hp.choice('n', np.arange(3,20, step =1))
#         }
                                
                                
                                
# best_classifier = fmin(objective_func, space, algo=tpe.suggest, max_evals=30)
# print(best_classifier)# With Bayesian Optimization, and Random Forest



In [ ]:
# x_val_lda = lda.transform(obama_val_pr_tfidf)
# x_val_lda_df = pd.DataFrame(data = x_val_lda)

In [ ]:
# c = lambda b : 1 if b == 0 else b
# d = lambda b : 2 if b <= 1 else b
# bc = RandomForestClassifier(max_depth = c(best_classifier['max_depth']), min_samples_leaf = c(best_classifier['min_samples_leaf']), min_samples_split = d(best_classifier['min_samples_split'])
#                             ,n_estimators = best_classifier['n_estimators'])

# bc.fit(x_lda2, obama_train2_pr_class)

# y_pred = bc.predict(x_val_lda)

# accuracy_score(obama_val_pr_class, y_pred)

In [ ]:
# lda_clf = lda.fit(obama_train2_pr_tfidf, obama_train2_pr_class)
# y_pred_lda_clf = lda_clf.predict(obama_val_pr_tfidf)
# # y_pred_lda_clf

# accuracy_score(obama_val_pr_class, y_pred_lda_clf)

### Light Gradient Boosting

In [ ]:
import lightgbm as lgbm
from lightgbm.sklearn import LGBMClassifier

In [ ]:
# With Bayesian Optimization, and Light GB

def objective_func_LGB(args):
    boosting_type= args['boosting_type']
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    learning_rate = args['learning_rate']
    subsample = args['subsample']
    min_split_gain = args['min_split_gain']

    par_dict = {'boosting_type': boosting_type, 'n_estimators': n_estimators, 'max_depth': max_depth, 'learning_rate': learning_rate, 'subsample': subsample, 'min_split_gain': min_split_gain}
    print(par_dict)
    
    clf = LGBMClassifier(boosting_type = boosting_type, class_weight  = 'balanced', num_leaves = 2**max_depth, n_estimators = n_estimators, max_depth = max_depth, subsample=subsample, learning_rate=learning_rate, min_split_gain = min_split_gain)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train_pr_tfidf,train_pr_df['Sentiment'], clf) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {
    'boosting_type' : hp.choice('boosting_type', ['gbdt', 'dart', 'goss']),
    'n_estimators': hp.choice('n_estimators', np.arange(50,1001, step =50)),
    'learning_rate':  hp.uniform('learning_rate', 0,0.2), 
    'max_depth': hp.choice('max_depth', np.arange(3, 10, step =1)),   
     'subsample': hp.choice('subsample', np.linspace(0.8, 1, num=4)),
    'min_split_gain': hp.choice('min_split_gain' , np.arange(0,6, step =1))}
                                
                                
                                
best_classifier_LGB = fmin(objective_func_LGB, space, algo=tpe.suggest, max_evals=25)
print(best_classifier_LGB)

{'boosting_type': 'goss', 'n_estimators': 900, 'max_depth': 9, 'learning_rate': 0.0005302970688930043, 'subsample': 0.9333333333333333, 'min_split_gain': 5}
{'accuracy': 0.506, 'f1_pos': 0.478, 'f1_neu': 0.396, 'f1_neg': 0.588, 'eval_score': 0.524}
{'boosting_type': 'dart', 'n_estimators': 900, 'max_depth': 7, 'learning_rate': 0.15031426232480716, 'subsample': 0.8666666666666667, 'min_split_gain': 5}
{'accuracy': 0.524, 'f1_pos': 0.503, 'f1_neu': 0.439, 'f1_neg': 0.597, 'eval_score': 0.541}
{'boosting_type': 'goss', 'n_estimators': 150, 'max_depth': 7, 'learning_rate': 0.19809633629294132, 'subsample': 0.9333333333333333, 'min_split_gain': 0}
{'accuracy': 0.517, 'f1_pos': 0.503, 'f1_neu': 0.453, 'f1_neg': 0.576, 'eval_score': 0.532}
{'boosting_type': 'goss', 'n_estimators': 750, 'max_depth': 5, 'learning_rate': 0.13876934680274372, 'subsample': 0.8, 'min_split_gain': 4}
{'accuracy': 0.525, 'f1_pos': 0.503, 'f1_neu': 0.46, 'f1_neg': 0.59, 'eval_score': 0.539}
{'boosting_type': 'gbdt', '

In [ ]:
# With PCA, Bayesian Optimization, and Light GB

def objective_func_LGB_PCA(args):
    boosting_type= args['boosting_type']
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    learning_rate = args['learning_rate']
#     subsample = args['subsample']
    min_split_gain = args['min_split_gain']
    n = args['n']
    
    par_dict = {'n': n, 'n_estimators': n_estimators, 'max_depth': max_depth, 'learning_rate': learning_rate, 'subsample': 1, 'min_split_gain': min_split_gain} 
    print(par_dict)
    
    clf = LGBMClassifier(boosting_type = boosting_type, class_weight  = 'balanced', num_leaves = 2**max_depth, n_estimators = n_estimators, max_depth = max_depth, subsample=1, learning_rate=learning_rate, min_split_gain = min_split_gain)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train_pr_tfidf,train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {
    'boosting_type' : hp.choice('boosting_type', ['gbdt', 'dart', 'goss']),
    'n_estimators': hp.choice('n_estimators', np.arange(50,1001, step =100)),
    'learning_rate':  hp.uniform('learning_rate', 0,0.2), 
    'max_depth': hp.choice('max_depth', np.arange(3, 10, step =1)),   
#      'subsample': hp.choice('subsample', np.linspace(0.8, 1, num=4)),
    'min_split_gain': hp.choice('min_split_gain' , np.arange(0,6, step =1)),
        'n': hp.choice('n', np.arange(3,203, step =20))}
                                
                                
                                
best_classifier_LGB_PCA = fmin(objective_func_LGB_PCA, space, algo=tpe.suggest, max_evals=35)
print(best_classifier_LGB_PCA)

{'n': 83, 'n_estimators': 150, 'max_depth': 6, 'learning_rate': 0.1960623520083565, 'subsample': 1, 'min_split_gain': 1}
{'accuracy': 0.506, 'f1_pos': 0.463, 'f1_neu': 0.436, 'f1_neg': 0.58, 'eval_score': 0.516}
{'n': 83, 'n_estimators': 750, 'max_depth': 5, 'learning_rate': 0.05088249025808522, 'subsample': 1, 'min_split_gain': 2}
{'accuracy': 0.543, 'f1_pos': 0.509, 'f1_neu': 0.474, 'f1_neg': 0.615, 'eval_score': 0.556}
{'n': 83, 'n_estimators': 150, 'max_depth': 7, 'learning_rate': 0.022378320037396683, 'subsample': 1, 'min_split_gain': 1}
{'accuracy': 0.533, 'f1_pos': 0.512, 'f1_neu': 0.454, 'f1_neg': 0.603, 'eval_score': 0.549}
{'n': 3, 'n_estimators': 250, 'max_depth': 4, 'learning_rate': 0.17885938403990365, 'subsample': 1, 'min_split_gain': 0}
{'accuracy': 0.451, 'f1_pos': 0.425, 'f1_neu': 0.398, 'f1_neg': 0.508, 'eval_score': 0.461}
{'n': 63, 'n_estimators': 650, 'max_depth': 3, 'learning_rate': 0.04678839325729607, 'subsample': 1, 'min_split_gain': 5}
{'accuracy': 0.528, 'f1_

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(sampling_strategy='not majority',random_state=42)
romney_train_pr_tfidf_smote, romney_train_pr_class = sm.fit_sample(romney_train_pr_tfidf, romney_train_pr_df['Sentiment'] )


sm = SMOTE(sampling_strategy='not majority',random_state=42)
obama_train_pr_tfidf_smote, obama_train_pr_class = sm.fit_sample(obama_train_pr_tfidf, obama_train_pr_df['Sentiment'])